In [526]:
#import gensim
from gensim import corpora, models, similarities, utils
from pprint import pprint   # pretty-printer
import logging
import numpy as np
import pandas as pd
import nltk
import string
import pymysql
import re
import pyLDAvis.gensim
import pyLDAvis
import KAsql2 as ka

In [527]:
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

NUM_TOPICS = 30

In [528]:
# from KAsql import query_SQL
sql = 'SELECT id,business_id,stars,comment FROM review;' 
rows = ka.query_SQL(sql)#returns a tuple within a tuple: each row is a tuple, inside one big tuple.

documents = []#type list
ndocs = np.shape(rows)[0]

#reviews = [[review for review in rows[3]] for row in rows]
for row in rows:
    documents.append(row[3])# row[3] is type string, documents is type list

USE ortho;


In [529]:
print(documents[0])# prints one review

Some background: I rolled my ankle and had a really nasty sprain on Sunday. I tried to R.I.C.E. but had this nagging feeling that I should get it checked out. I had gone with a friend's recommendation with a doc but she was on vacation for 2 weeks and her assistant couldn't offer an honest recommendation to another doctor. Sooo here's how Yelp comes into play. Dr Chen has great rates so why not. His friendly staff sets me up with an appointment same day after I explain my situation. I show up and this is definitely a busy office. For people who complain about waiting past their appointment time: Well... don't see a populargood doctor. There's a reason everyone is waiting. I use to work in doctor offices and they do try their best to stay on time but emergencies come up. Okay, moving on. There were some unknowns about my insurance plan but guess what: you pay for medical care. I care that my doctor is competent, has good bedside manners, and isn't recommending outrageous surgeries. Dr. 

In [530]:
def canonicalize(word):    
    bads = ['dr.','dr',"n't","'ve","'re",'wo',"'m"]
    goods = ['doctor','doctor','not','have','are','will','am']

    try:
        idx = bads.index(word)
        word=goods[idx]
    except ValueError as e:
        word=word
    return word

In [531]:
def unify_pronouns(word):
    """ map many onto one for pronouns - parts of speech"""
    many = ['he',"his","him","she",'her',"hers",
            'I','i','me','mine','my',"i'm","i've",
            'you','your','yours',"you're","you've",
            'it','its',"it's",
            'they','their','theirs',"they're","they've",
            'we','us','our','ours',"we're","we've"]
    few = ['male','male','male','female','female','female',
           'firstper','firstper','firstper','firstper','firstper','firstper','firstper',
           'secper','secper','secper','secper','secper',
           'neut3rd','neut3rd','neut3rd',
           'neut3rd','neut3rd','neut3rd','neut3rd','neut3rd',
          'coll3rd','coll3rd','coll3rd','coll3rd','coll3rd','coll3rd']
    try:
        idx = many.index(word)
        word=few[idx]
    except ValueError as e:
        word=word
    return word

In [532]:
def numwords():
    #add numbers to common words
    nums = np.arange(0,100)
    ss =''
    ss = ss.join([' '+str(n) for n in nums])
    return ss

In [533]:
def ids_to_names():
# Purpose of function is to exclude proper names of doctors - first and last names
# Function will accept list of yelp_ids extracted by mysql from business table
# Function will put together a list of names
# Function will exclude terms like family doctor and pediatrics

    dashed_names=[]#examples = ['tamalpais-pediatrics-novato','julie-doctor-kim']
    sql = 'SELECT DISTINCT(yelp_id) FROM business;' 
    yelp_ids = ka.query_SQL(sql)# extracts unique yelp_ids
    for i in range(0,len(yelp_ids)):
        dashed_names.append(yelp_ids[i][0])
    
    def split_append(yelp_id):
        yelp_id = yelp_id.split('-')
        #print(yelp_id)
        return yelp_id

#     business_type = ['pediatrics','family','medicine','doctor','institute','center',
#                      'practice','md','san','francisco','ucsf','one','medical','group',
#                     'sports','fitness','therapy','acupuncture','chiropractic','district']
    
    names = []
#     for name in dashed_names:
#         [names.append(l) for l in split_append(name) if l not in business_type]
    for name in dashed_names:
        [names.append(l) for l in split_append(name)]
    return names

In [534]:
def get_pat_for_tokenizer():
    from nltk.tokenize import regexp_tokenize
    """money and time patterns formatted to fit in nltk RegexpTokenizer object, debugged with s2 below"""
    #s2 = 'it took until 12:30 am to get an appointment and I spent $3.50'
    
    pat = '\w+|\$[\d\.]+|\S+'
    pat2 = '\d+\:[\d]+\s?(pm|am)|\S+'

    pat3 = '('+pat2+')' +'|'+ '('+pat+')'#must be one string argument... | is OR
    #tokenizer = RegexpTokenizer(pat3)#time
    return pat3

In [535]:
# Develop another exclude list for certain pronouns, which is a separate layer from what i previously did with stopwords

def exclude_pronoun_list(texts):

    pos_list = [nltk.pos_tag(word) for word in texts]
    #print pos_list[:][1]
    
    index=0
    pos_dict={}

    for rev in pos_list:#rev type list
        for tup in rev:
            index +=1
    #         if index>5:
    #             break
            #print 'count: ',count, 'tup: ', tup[0], tup[1]
            pos_dict[index] = {'word':tup[0],'pos':tup[1]}

    pos_exclude = pd.DataFrame.from_dict(pos_dict.values(),orient='columns')
    #print pos_exclude.shape
    #print pos_exclude.head(5)
    
    # RB = AVB; CD = Cardinal Number; PRP/PRP$ = personal pronoun; RB=ADVERB; IN = Preposition
    JJ = pos_exclude['pos']=='JJ'#adjective
    RB = pos_exclude['pos']=='RB'#adverb
    CD = pos_exclude['pos']=='CD'#cardinal number
    PRP = pos_exclude['pos']=='PRP'#personal pronoun
    PRPD = pos_exclude['pos']=='PRP$'#possessive personal pronoun
    IN = pos_exclude['pos']=='IN'#preposition

    # exclude_list = JJ+CD# This didn't work for some reason
    exclude_list = pos_exclude.loc[RB]['word'].tolist()#series to list
    exclude_list = exclude_list+pos_exclude.loc[CD]['word'].tolist()
    exclude_list = exclude_list+pos_exclude.loc[PRP]['word'].tolist()
    exclude_list = exclude_list+pos_exclude.loc[PRP]['word'].tolist()
    exclude_list = exclude_list+pos_exclude.loc[IN]['word'].tolist()
    #print exclude_list

    exc_list = [unicode(word) for word in exclude_list]
    print exc_list[:100]

    
    return exc_list

In [536]:
# CLEAN & TOKENIZE - stoplists, lowercase, regular expressions, time-expressions

import string

# from nltk.corpus import stopwords
# from nltk.tokenize import RegexpTokenizer

#texts: A list of lists; text: list of tokenized words from 1 doc/review
#document = a string

prevalent = ['dr.','dr','doc','doctor','doctors',"doctor's","doctors'",'get','see','saw','could','would','have','had','did','yelp','try',
                "there's",'ever',"here's",'here',"couldn't",'go','got','thing','things','made',"don't",'do','done','made','make',
                "it's",'went','\&','&','went','one',"he's","she's","he'd","she'd","i'd","i'm",'it',",","it's"]#type list
surgery = ['surgery','surgical','surgury','surgeon','surgeons','surgeries']

#tokenizer = RegexpTokenizer('\w+|\$[\d\.]+|\S+')#money
#tokenizer = RegexpTokenizer('\d*'+'\:'+'\d*')#time
tokenizer = RegexpTokenizer(get_pat_for_tokenizer())#time & money
mystopwords = stopwords.words() + ids_to_names() + prevalent + surgery
#print(mystopwords[-500:])

# texts = [[canonicalize(word) for word in tokenizer.tokenize(document.lower().replace('.',' ')) if word not in mystopwords]
#         for document in documents]#review in reviews 

texts = [[canonicalize(word) for word in tokenizer.tokenize(document.lower().translate(None, string.punctuation)) if word not in mystopwords]
        for document in documents]#review in reviews 


USE ortho;


In [537]:
exc_list = exclude_pronoun_list(texts)

[u'really', u'friendly', u'definitely', u'well', u'really', u'definitely', u'incredibly', u'particularly', u'rather', u'thoroughly', u'accurately', u'mannereven', u'still', u'often', u'nearly', u'truly', u'surprisingly', u'always', u'friendly', u'personally', u'however', u'splendidly', u'still', u'ten', u'ago', u'nearly', u'weekly', u'daily', u'already', u'fully', u'confidently', u'much', u'even', u'rather', u'extremely', u'prior', u'potentially', u'potentially', u'fortunately', u'back', u'back', u'back', u'prior', u'well', u'still', u'extremely', u'early', u'back', u'completely', u'ago', u'right', u'back', u'fast', u'forward', u'right', u'ago', u'first', u'really', u'well', u'track', u'even', u'back', u'enough', u'quickly', u'well', u'really', u'really', u'much', u'back', u'much', u'really', u'right', u'incredibly', u'barely', u'wellalso', u'sometimes', u'much', u'else', u'always', u'easily', u'directly', u'often', u'possibly', u'positively', u'regularly', u'additionally', u'highly', 

In [538]:
print exc_list[:60]

[u'really', u'friendly', u'definitely', u'well', u'really', u'definitely', u'incredibly', u'particularly', u'rather', u'thoroughly', u'accurately', u'mannereven', u'still', u'often', u'nearly', u'truly', u'surprisingly', u'always', u'friendly', u'personally', u'however', u'splendidly', u'still', u'ten', u'ago', u'nearly', u'weekly', u'daily', u'already', u'fully', u'confidently', u'much', u'even', u'rather', u'extremely', u'prior', u'potentially', u'potentially', u'fortunately', u'back', u'back', u'back', u'prior', u'well', u'still', u'extremely', u'early', u'back', u'completely', u'ago', u'right', u'back', u'fast', u'forward', u'right', u'ago', u'first', u'really', u'well', u'track']


In [539]:
print texts[:2]

[['background', 'rolled', 'ankle', 'really', 'nasty', 'sprain', 'sunday', 'tried', 'rice', 'nagging', 'feeling', 'checked', 'gone', 'friends', 'recommendation', 'vacation', 'weeks', 'assistant', 'couldnt', 'offer', 'honest', 'recommendation', 'another', 'sooo', 'heres', 'comes', 'play', 'great', 'rates', 'friendly', 'staff', 'sets', 'appointment', 'day', 'explain', 'situation', 'show', 'definitely', 'busy', 'office', 'people', 'complain', 'waiting', 'past', 'appointment', 'time', 'well', 'dont', 'populargood', 'theres', 'reason', 'everyone', 'waiting', 'use', 'work', 'offices', 'best', 'stay', 'time', 'emergencies', 'okay', 'moving', 'unknowns', 'insurance', 'plan', 'guess', 'pay', 'medical', 'care', 'care', 'competent', 'good', 'bedside', 'manners', 'isnt', 'recommending', 'outrageous', 'warm', 'funny', 'sure', 'knew', 'getting', 'take', 'care', 'granted', 'insurance', 'network', 'id', 'pay', 'nominal', 'fee', 'think', 'says', 'lot', 'fee', 'super', 'fair', 'insurance', 'tough', 'chen

In [543]:
newtexts = [word for word in texts if word not in exc_list]
# if [(exc in text for exc in exc_list) for text in texts]:
#     print exc
#print newtexts[:2]# exclusion above is not working

In [541]:
alpha = ['really','friendly','definitely']
b = ['friendly']

# for a in alpha:
#     print a

if (b in a for a in alpha):
    print b
    a.drop(b)
print a
    


['friendly']


AttributeError: 'str' object has no attribute 'drop'

In [257]:
# Context
#result = [nltk.pos_tag(text) for text in texts[:2]]
#print result

print texts[1]
simtext = nltk.Text(texts) 

['performed', 'acl', 'meniscus', 'repair', 'surgery', 'knee', '52014', 'entire', 'process,', 'staff', 'incredibly', 'professional', 'supportive,', 'particularly', 'helping', 'rather', 'complex', 'insurance', 'process', 'took', 'time', 'answer', 'questions', 'thoroughly', 'accurately,', 'great', 'bedside', 'manner', 'even', 'left', 'francisco,', 'still', 'responded', 'emails,', 'often', 'within', '24', 'hours', 'another', 'exchange', 'yesterday', 'nearly', '13', 'months', 'postop!this', 'surgery', 'hardest', "i've", 'faced,', 'feel', 'truly', 'lucky', 'help', 'thanks', 'help!']


In [258]:
#print(simtext.similar('acl')) 
#print(simtext, np.shape(simtext))
#print(simtext.similar('pain'))
#simtext.similar('nasty')

In [259]:
# Parts of Speech
result = [nltk.pos_tag(text) for text in texts[:2]]
#nltk.help.upenn_tagset('RB')
#print result

In [260]:
# BAG OF WORDS

# prevalent = ['dr.','dr','doc','doctor','doctors',"doctor's","doctors'",'get','see','saw','could','would','have','had','did','yelp','try',
#                 "there's",'ever',"here's",'here',"couldn't",'go','got','thing','things','made',"don't",'do','done','made','make',
#                 "it's",'went','\&','&','went','surgery','surgical','surgury','surgeon','surgeons','surgeries','one',"he's","she's",
#                  "he'd","she'd","i'd","i'm",'it',",","it's"]#type list

dictionary = corpora.Dictionary(texts)# create dictionary object from cleaned texts
#dictionary.filter_extremes(no_below=10, no_above=.67)#less than 10 documents (absolute number)/ more than fraction of size
#dictionary.filter_tokens(bad_ids=prevalent)
dictionary.compactify()

dictionary.save('/Users/kaschbacher/insightproject/yelp/git-yelp/orthopedists.dict') # store the dictionary, for future reference
print(dictionary)
#print(type(dictionary))#<class 'gensim.corpora.dictionary.Dictionary'>
#print(dictionary.token2id)# very long!

# To merge with another dictionary -  Dictionary.merge_with())# Apply bag of words to reviews & print tokenized dictionary

Dictionary(7632 unique tokens: [u'cussed', u'reconstruction,', u'yellow', u'four', u"friend's"]...)


In [261]:
# SERIALIZE & SAVE TO DISK 

# Convert tokenized documents to sparse vectors: -- [(0, 1), (4, 3),...
# new_vec = dictionary.doc2bow(documents[1].lower().split())
# print(new_vec)

corpus = [dictionary.doc2bow(text) for text in texts]
corpus_mm = corpora.MmCorpus.serialize('orthopedists_bow.mm', corpus) # store to disk, for later use

# corpus looks like - [[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1)... (35, 2)...
# (word order index, frequency of occurence?)

In [262]:
# LDA  (takes awhile to run)
# The constructor estimates Latent Dirichlet Allocation model parameters based on a training corpus 
# id2word is a mapping from word ids (integers) to words (strings). 

# 1) The constructor - obtains LDA model parameters given training corpus
lda = models.ldamodel.LdaModel(corpus, id2word=dictionary, num_topics=NUM_TOPICS, update_every=0, passes=10, iterations=100)


In [263]:
# SAVE THE LDA OBJECT
# --- note - i tried to do this with gensim, but it wasn't working, and ultimately cpickle did
#from gensim.utils import SaveLoad
#save(fname_or_handle, separately=None, sep_limit=10485760, ignore=frozenset([]), pickle_protocol=2)
#utils.SaveLoad.save(lda, pickle_protocol=2)

import cPickle as pickle
pickle_out = open('lda_ortho_'+str(NUM_TOPICS)+'t_filter.p', 'wb')
pickle.dump(lda, pickle_out)
pickle_out.close()

In [264]:
# PRINT THE TOPICS (list of tuples)
#lda.top_topics(corpus,num_words=6)

In [274]:
# TOPIC DISTRIBUTIONS MATRIX
# Form a matrix size R by T (Reviews by Topics). Columns = topic number (30). Elements = topic_dist.
# Kirstin

num_reviews = np.shape(corpus)[0]
zeros = np.zeros([num_reviews,NUM_TOPICS])
col_names = np.arange(0,30)
R_topic_dist = pd.DataFrame(zeros,columns = col_names)

count=0
for rev_bow in corpus:
    # For debugging
#     if count >10:
#         break
    topic_dist = lda[rev_bow] 
    #print 'row',count,'topic dist',topic_dist
    for topic_tup in topic_dist:
        mytopic = topic_tup[0]
        myvalue = topic_tup[1]
        R_topic_dist.loc[count,mytopic]=myvalue
        #print topic_tup[0], topic_tup[1]
    count+=1
    
#print R_topic_dist.loc[30:32,10:17]

In [276]:
lda.print_topics(10)

[u"0.011*surgery + 0.011*minutes + 0.011*hip + 0.008*staff + 0.007*appointment + 0.007*didn't + 0.006*time + 0.006*surgeon + 0.006*first + 0.006*waiting",
 u'0.011*knee + 0.010*office + 0.009*said + 0.008*back + 0.007*surgery + 0.007*staff + 0.006*time + 0.006*recommend + 0.006*weeks + 0.006*first',
 u'0.012*time + 0.011*pain + 0.007*surgeon + 0.007*back + 0.007*ankle + 0.007*even + 0.006*hip + 0.006*problem + 0.006*best + 0.006*good',
 u'0.009*care + 0.008*office + 0.007*surgery + 0.007*patient + 0.007*like + 0.007*time + 0.007*staff + 0.006*front + 0.006*people + 0.005*treated',
 u"0.014*pain + 0.012*great + 0.009*back + 0.009*physical + 0.008*didn't + 0.008*good + 0.008*surgery + 0.007*shoulder + 0.006*going + 0.006*super",
 u'0.020*surgery + 0.011*knee + 0.009*care + 0.009*pain + 0.008*shoulder + 0.007*good + 0.007*office + 0.007*time + 0.006*mri + 0.006*surgeon',
 u'0.016*surgery + 0.011*great + 0.008*pain + 0.007*recommend + 0.007*highly + 0.006*like + 0.006*never + 0.006*staff +

In [277]:
# Add the R-ID (primary key) and B-ID (foreign key) to the NORM-GAMMAS DF
sql = 'SELECT id,business_id FROM review'+';'
rows = ka.query_SQL(sql)# returns shape #reviews x 2

ID_df = pd.DataFrame(np.array(rows),columns=['RID','BID'])
Review_topics_df = pd.concat([ID_df,R_topic_dist],axis=1)
#print Review_topics_df.ix[2000:2005,10:35]
#print Review_topics_df.head(5)

print R_topic_dist.tail(5)
rev_sums = R_topic_dist.sum(axis=1,skipna=True)
print 'shape of rev sums: ',np.shape(rev_sums)
print R_topic_dist.shape
print np.min(rev_sums)
x=np.where(rev_sums==0)#column indices for 
print 'shape: ',np.shape(x)#number of rows over which sum of topics is 0, meaning LDA not applied to those rows

USE ortho;
            0   1         2   3   4   5   6   7   8   9  ...  20  21  \
7315  0.000000   0  0.000000   0   0   0   0   0   0   0 ...   0   0   
7316  0.000000   0  0.966667   0   0   0   0   0   0   0 ...   0   0   
7317  0.000000   0  0.000000   0   0   0   0   0   0   0 ...   0   0   
7318  0.983333   0  0.000000   0   0   0   0   0   0   0 ...   0   0   
7319  0.000000   0  0.000000   0   0   0   0   0   0   0 ...   0   0   

            22  23  24  25  26  27  28  29  
7315  0.000000   0   0   0   0   0   0   0  
7316  0.000000   0   0   0   0   0   0   0  
7317  0.000000   0   0   0   0   0   0   0  
7318  0.000000   0   0   0   0   0   0   0  
7319  0.956061   0   0   0   0   0   0   0  

[5 rows x 30 columns]
shape of rev sums:  (7320,)
(7320, 30)
0.879166666626
shape:  (1, 0)


In [269]:
# SAVE THE NORM_GAMMAS DATAFRAME OBJECT

import cPickle as pickle

pickle_out = open('R_topic_dist.p', 'wb')#This df just has topic distributions
pickle.dump(R_topic_dist, pickle_out)
pickle_out = open('Review_topics_df.p', 'wb')#This df has RID and BID in front of the topic distributions
pickle.dump(Review_topics_df, pickle_out)
pickle_out.close()

In [546]:
# example code for grouping by business

bid_temp = Review_topics_df.drop('RID', 1)
bid_tmeans = bid_temp.groupby('BID',axis=0,as_index='BID').mean()
bid_tmeans = bid_tmeans.sort([1,2,3,4,5],ascending=False)

In [550]:
print bid_tmeans.head(10)
#print bid_tmeans.ix[54,:]

     0         1   2         3   4         5         6   7         8   9   \
BID                                                                         
54    0  0.098176   0  0.099113   0  0.096778  0.098761   0  0.097752   0   
34    0  0.098176   0  0.099113   0  0.096778  0.098761   0  0.097752   0   
234   0  0.098176   0  0.099113   0  0.096778  0.098761   0  0.097752   0   
254   0  0.098176   0  0.099113   0  0.096778  0.098761   0  0.097752   0   
74    0  0.098176   0  0.099113   0  0.096778  0.098761   0  0.097752   0   
274   0  0.098176   0  0.099113   0  0.096778  0.098761   0  0.097752   0   
134   0  0.098176   0  0.099113   0  0.096778  0.098761   0  0.097752   0   
94    0  0.098176   0  0.099113   0  0.096778  0.098761   0  0.097752   0   
194   0  0.098176   0  0.099113   0  0.096778  0.098761   0  0.097752   0   
14    0  0.098176   0  0.099113   0  0.096778  0.098761   0  0.097752   0   

       ...     20        21  22        23  24  25  26  27  28        29  
B

In [551]:
#SAVE (ADD to .GITIGNORE)
pickle_out = open('bid_tmeans.p', 'wb')
pickle.dump(bid_tmeans, pickle_out)
pickle_out.close()

In [271]:
vis_data = pyLDAvis.gensim.prepare(lda, corpus, dictionary)
pyLDAvis.display(vis_data)#html object

In [552]:
#Save figure object. 
pickle.dump(vis_data, open('vis_data_30t.p','wb'))